### Route comparison tutorial

This tutorial shows how to read routes from RDF files, assign atom-mapping, and then compute route similarity.

We will use three experimental routes of atovastatin as examples

In [ ]:
import os
# To use rxnmapper, this assumes your conda and the 
# environments are installed in your home directory - update if necessary
conda_path = os.path.expanduser("~") + "/miniconda3"
os.environ["CONDA_PATH"] = f"{conda_path}/bin/"
os.environ["RXNMAPPER_ENV_PATH"] = f"{conda_path}/envs/rxnmapper/"

import json
import glob

from rxnutils.routes.readers import read_rdf_file
from rxnutils.routes.comparison import simple_route_similarity

We will read each of the RDF files in reverse alphabetical order

In [ ]:
rdf_filenames = glob.glob("*.rdf")
rdf_filenames.sort(reverse=True)
len(rdf_filenames)

For each RDF, we will:

* Read the RDF to a `SynthesisRoute` object using `read_rdf_file`
* Assign atom-mapping with `assign_atom_mapping` (here we will only use `rxnmapper` tool)
* If this is the 2:nd or 3:rd route, re-map the route to correspond to the 1:st route
* Save a PNG with the route showing the atom-mapping
* Save the route in JSON format

In [ ]:
ref_route = None
routes = []
for idx, filename in enumerate(rdf_filenames):
    imgfilename = filename.replace(".rdf", ".png")
    print(filename, flush=True)
    route = read_rdf_file(filename)
    route.assign_atom_mapping(only_rxnmapper=True, overwrite=True)
    if idx == 0:
        ref_route = route
    else:
        route.remap(ref_route)
    route.image(show_atom_mapping=True).save(imgfilename)
    with open(filename.replace(".rdf", "_tree.json"), "w") as fileobj:
        json.dump(route.reaction_tree, fileobj, indent=2)
    routes.append(route)

Here is the first route

In [ ]:
routes[0].image(show_atom_mapping=True)

Now we can compute the pairwise route similarity between the three routes

In [ ]:
simple_route_similarity(routes)